<a href="https://colab.research.google.com/github/scharnk/madam_marie/blob/main/BiddyTarot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import seaborn as sns
sns.set()

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import requests
# import requests_cache
import dill
from bs4 import BeautifulSoup
from datetime import datetime
from retrying import retry
import re
import spacy
import itertools  # itertools.combinations may be useful
import networkx as nx

In [3]:
url_list = ['https://www.biddytarot.com/tarot-card-meanings/major-arcana/', 
            'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-cups/',
            'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-swords/',
            'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-pentacles/',
            'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-wands/']

In [44]:
url_major_arcana = [
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/fool/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/magician/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/high-priestess/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/empress/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/emperor/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/hierophant/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/lovers/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/chariot/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/strength/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/hermit/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/wheel-of-fortune/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/justice/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/hanged-man/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/death/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/temperance/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/devil/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/tower/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/star/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/moon/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/sun/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/judgement/',
                    'https://www.biddytarot.com/tarot-card-meanings/major-arcana/world/'
]

In [45]:
url_minor_arcana = [
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-cups/ace-of-cups/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-cups/two-of-cups/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-cups/three-of-cups/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-cups/four-of-cups/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-cups/five-of-cups/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-cups/six-of-cups/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-cups/seven-of-cups/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-cups/eight-of-cups/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-cups/nine-of-cups/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-cups/ten-of-cups/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-cups/page-of-cups/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-cups/knight-of-cups/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-cups/queen-of-cups/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-cups/king-of-cups/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-swords/ace-of-swords/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-swords/two-of-swords/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-swords/three-of-swords/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-swords/four-of-swords/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-swords/five-of-swords/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-swords/six-of-swords/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-swords/seven-of-swords/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-swords/eight-of-swords/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-swords/nine-of-swords/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-swords/ten-of-swords/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-swords/page-of-swords/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-swords/knight-of-swords/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-swords/queen-of-swords/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-swords/king-of-swords/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-pentacles/ace-of-pentacles/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-pentacles/two-of-pentacles/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-pentacles/three-of-pentacles/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-pentacles/four-of-pentacles/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-pentacles/five-of-pentacles/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-pentacles/six-of-pentacles/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-pentacles/seven-of-pentacles/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-pentacles/eight-of-pentacles/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-pentacles/nine-of-pentacles/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-pentacles/ten-of-pentacles/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-pentacles/page-of-pentacles/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-pentacles/knight-of-pentacles/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-pentacles/queen-of-pentacles/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-pentacles/king-of-pentacles/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-wands/ace-of-wands/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-wands/two-of-wands/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-wands/three-of-wands/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-wands/four-of-wands/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-wands/five-of-wands/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-wands/six-of-wands/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-wands/seven-of-wands/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-wands/eight-of-wands/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-wands/nine-of-wands/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-wands/ten-of-wands/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-wands/page-of-wands/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-wands/knight-of-wands/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-wands/queen-of-wands/',
                    'https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-wands/king-of-wands/'
]

In [6]:
card_list = [
             "Fool",
             "The Magician",
             "High Priestess",
             "The Empress",
             "The Emperor",
             "Hierophant",
             "Lovers",
             "Chariot",
             "Strength",
             "Hermit",
             "Wheel of Fortune",
             "Justice",
             "Hanged Man",
             "Death",
             "Temperance",
             "Devil",
             "Tower",
             "Star",
             "Moon",
             "Sun",
             "Judgement",
             "World"
             ]

In [ ]:
for i, v in enumerate(url_major_arcana):
    print(i)

In [41]:
#run through url_major_arcana to get full card meanings for upright and reversed

major_meanings = []

for idx, url in enumerate(url_major_arcana):
    card = requests.get(url_major_arcana[idx])
    soup = BeautifulSoup(card.text, "lxml")
    p = soup.find_all(id = ["upright", "reversed"])
    text_p = []
    for i in p:
        text_p.append(i.getText())
    major_meanings.append(text_p)

#run through url_minor_arcana to get full card meanings for upright and reversed

minor_meanings_upright = []
minor_meanings_reversed = []

for idx, url in enumerate(url_minor_arcana):
    card = requests.get(url_minor_arcana[idx])
    soup = BeautifulSoup(card.text, "lxml")
    p = soup.find_all('div', {'class':'col span_12'})
    q = soup.find_all('div', {'class':'col span_8'})
    text_p = []
    text_q = []
    for i in p:
        # p.find('class'='col span_12')
        text_p.append(i.getText())
    minor_meanings_upright.append(text_p[-1])
    for i in q:
        text_q.append(i.getText())
    minor_meanings_reversed.append(text_q[-1])


#run through url list to get keywords for each card upright and reversed


#can figure out how to get general card descriptions later on... should be similar to card meanings




In [52]:
import pickle

In [53]:
# Save meanings so you don't have to re-scrape biddytarot.com

with open('major_meanings.pkl', 'wb') as f:
     pickle.dump(major_meanings, f)

with open('minor_meanings_upright.pkl', 'wb') as f:
     pickle.dump(minor_meanings_upright, f)

with open('minor_meanings_reversed.pkl', 'wb') as f:
     pickle.dump(minor_meanings_reversed, f)

In [54]:
# Read the meanings from the dill/pickle files above

with open('major_meanings.pkl', 'rb') as f:
     major_meanings = pickle.load(f)

with open('minor_meanings_upright.pkl', 'rb') as f:
     minor_meanings_upright = pickle.load(f)

with open('minor_meanings_reversed.pkl', 'rb') as f:
     minor_meanings_reversed = pickle.load(f)

['Upright HierophantThe Hierophant card represents an established set of spiritual values and beliefs and is often correlated with religion and other formal doctrines. Before you can discover your own belief systems and make your own choices (as associated with the next card, the Lovers), the Hierophant encourages you to learn the fundamental principles from a trusted source.Work with a teacher, mentor, or guide to teach you about spiritual values and beliefs in a structured way. He may be an authority or a kind and generous mentor who nurtures your spiritual awareness and helps you access the Divine by understanding the traditions and core principles. You may also undertake a period of formal study as you delve into a subject that has been widely explored and documented.If you have already mastered a particular field of study, you may be taking on the role of teacher and mentor to others. In this position, you honour and acknowledge your responsibility to share your knowledge in a str

In [46]:
# minor_meanings = []
# text_p = []


In [39]:
len(minor_meanings_upright)

56

In [ ]:
minor_meanings_upright

In [49]:
len(minor_meanings_reversed)

56

In [15]:
# major_meanings[3]

['Upright The EmpressThe Empress signifies a strong connection with our femininity. Femininity translates in many ways – elegance, sensuality, fertility, creative expression, nurturing – and is necessary for creating balance in both men and women. The Empress calls on you to connect with your feminine energy. Create beauty in your life. Connect with your senses through taste, touch, sound, smell and sight. Draw on these senses to experience pleasure and deep fulfilment. Treat yourself to a day spa, learn massage, enjoy a fine restaurant or spend more time with your partner. Discover different approaches to expressing yourself creatively, be it through painting, music, drama or other art forms. This may be the perfect chance to take up a new hobby that enables you to access this part of yourself.The Empress signifies abundance. You are surrounded by life’s pleasures and luxuries and have everything you need to live a comfortable lifestyle. You are in a period of growth, in which all you

In [ ]:
page = requests.get('https://www.biddytarot.com/tarot-card-meanings/') # Use requests.get to download the page.

In [ ]:
soup = BeautifulSoup(page.text, "html.parser")

In [ ]:
# url_base = 'https://www.biddytarot.com/'
# url_end = ['/tarot-card-meanings/major-arcana/', '/tarot-card-meanings/minor-arcana/suit-of-cups/', '/tarot-card-meanings/minor-arcana/suit-of-swords/', '/tarot-card-meanings/minor-arcana/suit-of-pentacles/', '/tarot-card-meanings/minor-arcana/suit-of-wands/']

In [ ]:
# soup.prettify

In [ ]:
page = requests.get(url_list[0]) # Use requests.get to download the page.
soup = BeautifulSoup(page.text, "html.parser")

In [ ]:
# for link in soup.find_all('a'):
#     print(link.get('href'))

In [ ]:
# def get_links(response):
#     soup = BeautifulSoup(response.text, "lxml")
#     links = soup.select('div[a]')
#     pairs = []
#     for i in range(len(links)):
#         pair = get_link_date(links[i])
#         pairs.append(pair)
#     return pairs # A list of URL, date pairs
#     return links
    # for li in div:
    # all_li = li.find_all('li')
    # for link in all_li.find_all('a'):
    #     print(link.get('href'))

In [ ]:
cups = requests.get('https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-cups/')
soup_cups = BeautifulSoup(cups.text,"lxml")
a = soup_cups.find_all('a')
for link in a:
    print(link.get('href'))

In [ ]:
swords = requests.get('https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-swords/')
soup_swords = BeautifulSoup(swords.text,"lxml")
a = soup_swords.find_all('a')
for link in a:
    print(link.get('href'))

In [ ]:
pentacles = requests.get('https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-pentacles/')
soup_pentacles = BeautifulSoup(pentacles.text,"lxml")
a = soup_pentacles.find_all('a')
for link in a:
    print(link.get('href'))

In [37]:
wands = requests.get('https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-wands/')
soup_wands = BeautifulSoup(wands.text,"lxml")
a = soup_wands.find_all('a')
for link in a:
    print(link.get('href'))

#ajax-content-wrap
https://biddytarot.com/tarot-guides/tarot-card-answers/
#
https://www.biddytarot.com
#searchbox
#sidewidgetarea
/btc-free-training/
https://community.biddytarot.com/free-tarot-readings/
/learn-tarot
/learn-tarot
/tarot-card-meanings
/tarot-101/
/community
/community
http://community.biddytarot.com/login-page/
/blog
/category/podcast
/shop
/business
/about
/about
https://www.biddytarot.com/careers/
/contact-faq
https://www.biddytarot.com/diversity/
/login
#searchbox
/tarot-card-meanings/
/tarot-card-meanings/minor-arcana/
https://biddytarot.com/tarot-guides/tarot-card-answers/
https://biddytarot.com/tarot-guides/tarot-card-answers/
https://biddytarot.com/tarot-guides/tarot-card-answers/
https://biddytarot.com/tarot-guides/tarot-card-answers/
https://www.biddytarot.com/archetype-quiz/
https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-wands/ace-of-wands/
https://www.biddytarot.com/tarot-card-meanings/minor-arcana/suit-of-wands/two-of-wands/
https://www

In [32]:
text_p = []
p = soup_fool.find_all(id = ["upright", "reversed"])
# text_p = [text_p.append(i.getText()) for i in p]
for i in p:
    text_p.append(i.getText())

In [24]:
p = soup_fool.find_all(id = ["upright", "reversed"])

In [33]:
text_p

['Upright FoolThe Fool is a card of new beginnings, opportunity and potential. Just like the young man, you are at the outset of your journey, standing at the cliff‘s edge, and about to take your first step into the unknown. Even though you don’t know exactly where you are going, you are being called to commit yourself and follow your heart, no matter how crazy this leap of faith might seem to you. Now is a time when you need to trust where the Universe is taking you.As you undertake this new journey, the Fool encourages you to have an open, curious mind and a sense of excitement. Throw caution to the wind and be ready to embrace the unknown, leaving behind any fear, worry or anxiety about what may or may not happen. This is about new experiences, personal growth, development, and adventure.The time is NOW! Take that leap of faith, even if you do not feel 100% ready or equipped for what is coming (who knows what it could be?!). Seriously, what are you waiting for? Do you think you need

In [ ]:
soup_fool.find_all('p')

In [18]:
a = soup_fool.find(id=["upright","reversed"])

In [36]:
major_page = requests.get('https://www.biddytarot.com/tarot-card-meanings/major-arcana/')

In [37]:
soup_major = BeautifulSoup(major_page.text, "lxml")

In [56]:
items = soup_major.find_all('div', {'class' : 'card_item'})

In [59]:
keywords = []
for i in items:
  keywords.append(i.getText())

In [60]:
keywords


['    UprightBeginnings, innocence, spontaneity, a free spiritReversedHolding back, recklessness, risk-takingFool   ',
 '    UprightManifestation, resourcefulness, power, inspired actionReversedManipulation, poor planning, untapped talentsThe Magician   ',
 '    UprightIntuition, sacred knowledge, divine feminine, the subconscious mindReversedSecrets, disconnected from intuition, withdrawal and silenceHigh Priestess   ',
 '    UprightFemininity, beauty, nature, nurturing, abundanceReversedCreative block, dependence on othersThe Empress   ',
 '    UprightAuthority, establishment, structure, a father figureReversedDomination, excessive control, lack of discipline, inflexibilityThe Emperor   ',
 '    UprightSpiritual wisdom, religious beliefs, conformity, tradition,institutionsReversedPersonal beliefs, freedom, challenging the status quoHierophant   ',
 '    UprightLove, harmony, relationships, values alignment, choicesReversedSelf-love, disharmony, imbalance, misalignment of valuesLovers

In [65]:
keywords_test = [i.split('Reversed') for i in keywords] 

In [66]:
keywords_test

[['    UprightBeginnings, innocence, spontaneity, a free spirit',
  'Holding back, recklessness, risk-takingFool   '],
 ['    UprightManifestation, resourcefulness, power, inspired action',
  'Manipulation, poor planning, untapped talentsThe Magician   '],
 ['    UprightIntuition, sacred knowledge, divine feminine, the subconscious mind',
  'Secrets, disconnected from intuition, withdrawal and silenceHigh Priestess   '],
 ['    UprightFemininity, beauty, nature, nurturing, abundance',
  'Creative block, dependence on othersThe Empress   '],
 ['    UprightAuthority, establishment, structure, a father figure',
  'Domination, excessive control, lack of discipline, inflexibilityThe Emperor   '],
 ['    UprightSpiritual wisdom, religious beliefs, conformity, tradition,institutions',
  'Personal beliefs, freedom, challenging the status quoHierophant   '],
 ['    UprightLove, harmony, relationships, values alignment, choices',
  'Self-love, disharmony, imbalance, misalignment of valuesLovers 

In [64]:
keywords3

['Holding back, recklessness, risk-takingFool   ',
 'Manipulation, poor planning, untapped talentsThe Magician   ',
 'Secrets, disconnected from intuition, withdrawal and silenceHigh Priestess   ',
 'Creative block, dependence on othersThe Empress   ',
 'Domination, excessive control, lack of discipline, inflexibilityThe Emperor   ',
 'Personal beliefs, freedom, challenging the status quoHierophant   ',
 'Self-love, disharmony, imbalance, misalignment of valuesLovers   ',
 'Self-discipline, opposition, lack of directionChariot   ',
 'Inner strength, self-doubt, low energy, raw emotionStrength   ',
 'Isolation, loneliness, withdrawalHermit   ',
 'Bad luck, resistance to change, breaking cyclesWheel of Fortune   ',
 'Unfairness, lack of accountability, dishonestyJustice   ',
 'Delays, resistance, stalling, indecisionHanged Man   ',
 'Resistance to change, personal transformation, inner purgingDeath   ',
 'Imbalance, excess, self-healing, re-alignmentTemperance   ',
 'Releasing limiting b

In [ ]:
print(items[0].getText())